# Energy Balance Comparison
### Author: Daniel Hogan
### Creation Date: July 29, 2024

This notebook will compare the net energy balance and accumulated total energy exchanges during winters 2022 and 2023 using data from SAIL, SPLASH, and SOS.

### Imports

In [26]:
# general
import os
import glob
import datetime as dt
import json
# data 
import xarray as xr 
from sublimpy import utils, variables, tidy, turbulence
import numpy as np
import pandas as pd
from act import discovery, plotting
# plotting
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# helper tools
from scripts.get_sail_data import get_sail_data
from scripts.helper_funcs import create_windrose_df, simple_sounding, mean_sounding
import scripts.helper_funcs as hf
from metpy import calc, units
# make plotly work 
init_notebook_mode(connected=True)
cf.go_offline()
# stats
import scipy.stats as stats
import seaborn as sns


### Data

In [19]:
##### 2022 data ####
winter_22 = slice('2021-12-01', '2022-03-31')
# SAIL data
# w22_sail_gts_met = xr.open_dataset('../../01_data/processed_data/sail_processed/sail_ds_1H_met_2022.nc').sel(time=winter_22)
w22_sail_kp_ecor = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_21_22/eddy_covariance_kettle_ponds_20211001_20220930.nc').sel(time=winter_22)
w22_sail_gts_sebs = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_21_22/sebs_20211001_20220930.nc').sel(time=winter_22)
# SPLASH data
w22_splash_kp_asfs = xr.open_dataset('../../01_data/processed_data/splash/w22_splash_kp_qc_1H.nc').sel(time=winter_22)
w22_splash_ap_asfs = xr.open_dataset('../../01_data/processed_data/splash/w22_splash_ap_qc_1H.nc').sel(time=winter_22)
w22_splash_kp_radsys = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/splash_data/radsys_ckp_dataset.nc').sel(time=winter_22)

##### 2023 data ####
winter_23 = slice('2022-12-01', '2023-03-31')
# SAIL data
# w23_sail_gts_met = xr.open_dataset('../../01_data/processed_data/sail_processed/sail_ds_1H_met_2023.nc').sel(time=winter_23)
w23_sail_kp_ecor = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_22_23/eddy_covariance_kettle_ponds_20221001_20230930.nc').sel(time=winter_23)
w23_sail_gts_sebs = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/sail_data/winter_22_23/sebs_20221001_20230930.nc').sel(time=winter_23)
# SPLASH data
w23_splash_ap_asfs = xr.open_dataset('../../01_data/processed_data/splash/w23_splash_ap_qc_1H.nc').sel(time=winter_23)
w23_splash_kp_radsys = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/splash_data/radsys_ckp_dataset.nc').sel(time=winter_23)
# SOS data
w23_sos_kp_all = xr.open_dataset('../../01_data/processed_data/sos_ds_1H_storage.nc').sel(time=winter_23)


In [81]:
# drop bad data for soil heat flux from SAIL
ds_for_soil = w22_sail_gts_sebs.where((w22_sail_gts_sebs['qc_surface_soil_heat_flux_1']==0) & (w22_sail_gts_sebs['qc_surface_soil_heat_flux_2']==0) & (w22_sail_gts_sebs['qc_surface_soil_heat_flux_3']==0))
w22_sail_gts_sebs['surface_soil_heat_flux_avg'] = (ds_for_soil.surface_soil_heat_flux_1 + ds_for_soil.surface_soil_heat_flux_2 + ds_for_soil.surface_soil_heat_flux_3)/3

ds_for_soil = w23_sail_gts_sebs.where((w23_sail_gts_sebs['qc_surface_soil_heat_flux_1']==0) & (w23_sail_gts_sebs['qc_surface_soil_heat_flux_2']==0) & (w23_sail_gts_sebs['qc_surface_soil_heat_flux_3']==0))
w23_sail_gts_sebs['surface_soil_heat_flux_avg'] = (ds_for_soil.surface_soil_heat_flux_1 + ds_for_soil.surface_soil_heat_flux_2 + ds_for_soil.surface_soil_heat_flux_3)/3

In [72]:
# calculate longwave radiation for SOS data
SIGMA = 5.67e-8
w23_sos_kp_all = w23_sos_kp_all.assign(R_lw_up = ((w23_sos_kp_all.Rpile_out_9m_d + (w23_sos_kp_all.Tcase_out_9m_d+273.15)**4*SIGMA)))
w23_sos_kp_all = w23_sos_kp_all.assign(R_lw_down = ((w23_sos_kp_all.Rpile_in_9m_d + (w23_sos_kp_all.Tcase_in_9m_d+273.15)**4*SIGMA)))

# calculate latent heat flux
Ls = 2.83e3 # J/kg
rho = 1000 # kg/m3
latent_heat_vars = [var for var in hf.TURBULENCE_VARIABLES if 'w_h2o__' in var]
for var in latent_heat_vars:
    new_var = var.replace('w_h2o__', 'LE_')
    w23_sos_kp_all = w23_sos_kp_all.assign({new_var: (w23_sos_kp_all[var]*Ls*rho/1000)})

# calculate sensible heat flux
gas_constant_dry_air = 287.058 # J/kg/K
density_of_air = (w23_sos_kp_all.P_10m_c*100)/(gas_constant_dry_air*(w23_sos_kp_all.T_10m_c+273.15))
heat_capacity_air = 1005 # J/kg/K
sensible_heat_vars = [var for var in hf.TURBULENCE_VARIABLES if 'w_tc_' in var]
for var in sensible_heat_vars:
    new_var = var.replace('w_tc__', 'H_')
    w23_sos_kp_all = w23_sos_kp_all.assign({new_var: (w23_sos_kp_all[var]*heat_capacity_air*density_of_air)})

For each dataset, let's build a dictionary of energy balance variables

In [86]:
energy_balance_variables = dict(
    SAIL= {'H':'sensible_heat_flux',
             'LE':'latent_heat_flux',
             'R_sw_down':'down_short_hemisp',
             'R_sw_up':'up_short_hemisp',
             'R_lw_down':'down_long',
             'R_lw_up':'up_long',
             'G':'surface_soil_heat_flux_avg',
             'R_net':'surface_energy_balance',},
    SPLASH= {'H':'Hs',
                'LE':'Hl',
                'R_sw_down':'down_short_hemisp',
                'R_sw_up':'up_short_hemisp',
                'R_lw_down':'down_long_hemisp',
                'R_lw_up':'up_long_hemisp',
                'G':['subsurface_heat_flux_A','subsurface_heat_flux_B'],
                'R_net':None,},
    SOS= {'H':'H_3m_c',
            'LE':'LE_3m_c',
            'R_sw_down':'Rsw_in_9m_d',
            'R_sw_up':'Rsw_out_9m_d',
            'R_lw_down':'R_lw_down',
            'R_lw_up':'R_lw_up',
            'G':None,
            'R_net':None,})

{'SAIL': {'H': 'sensible_heat_flux',
  'LE': 'latent_heat_flux',
  'R_sw_down': 'down_short_hemisp',
  'R_sw_up': 'up_short_hemisp',
  'R_lw_down': 'down_long',
  'R_lw_up': 'up_long',
  'G': 'surface_soil_heat_flux_avg',
  'R_net': 'surface_energy_balance'},
 'SPLASH': {'H': 'Hs',
  'LE': 'Hl',
  'R_sw_down': 'down_short_hemisp',
  'R_sw_up': 'up_short_hemisp',
  'R_lw_down': 'down_long_hemisp',
  'R_lw_up': 'up_long_hemisp',
  'G': ['subsurface_heat_flux_A', 'subsurface_heat_flux_B'],
  'R_net': None},
 'SOS': {'H': 'H_3m_c',
  'LE': 'LE_3m_c',
  'R_sw_down': 'Rsw_in_9m_d',
  'R_sw_up': 'Rsw_out_9m_d',
  'R_lw_down': 'R_lw_down',
  'R_lw_up': 'R_lw_up',
  'G': None,
  'R_net': None}}